### Import the Libraries and load the unscaled Dataset

In [1]:
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml
import pandas as pd
import numpy as np

df = pd.read_csv('mySet.csv')
df.head()

,Name,Magnetic core,Surface,Overall size (nm),Relaxivity (s-1/mmol Fe)( r2),B0 (T),Fe/cell (pg),cell viability
0,Fe2O3-PLL,1,PLL,15.0,NaN,1.5,64.51,0
1,uncoated,1,uncoated,10.0,549.0,4.7,29.30,1
2,D-mannose-coated Fe2O3,1,D-mannose,2.0,509.0,4.7,21.10,1
3,Fe2O3-PLL,1,PLL,10.0,492.0,4.7,24.50,1
4,PDMAAm-coated-Fe2O3,1,PDMMAm,10.0,89.0,4.7,23.20,1


We will drop the columns that we do not need for our model

In [2]:
for col in df.drop(['Magnetic core','Overall size (nm)','cell viability'], axis=1).columns:
     df.drop(col,axis=1,inplace=True)

In [3]:
df.head()

,Magnetic core,Overall size (nm),cell viability
0,1,15.0,0
1,1,10.0,1
2,1,2.0,1
3,1,10.0,1
4,1,10.0,1


### Instantiate and fit a scaling object

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)
scaled_data = pd.DataFrame(index=range(16), columns=df.columns, data=scaler.transform(df))

c:\users\user\desktop\test-jupyter\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [5]:
scaled_data.head()

,Magnetic core,Overall size (nm),cell viability
0,1.0,0.087838,0.0
1,1.0,0.054054,1.0
2,1.0,0.000000,1.0
3,1.0,0.054054,1.0
4,1.0,0.054054,1.0


### Split the Dataset on Train and Test according to the Kennard-Stone algorithm

In [6]:
train = df.drop(index=[2,4,8,9,10],axis = 0).copy()
train.drop('cell viability',axis = 1 , inplace=True)
train_labels = df['cell viability'].drop(index=[2,4,8,9,10],axis = 0).copy()

In [7]:
test = df.loc[[2,4,8,9,10]].drop('cell viability',axis=1).copy()
test_labels = df.iloc[[2,4,8,9,10]]['cell viability'].copy()

### Instantiate and fit a Logistic Regression classifier

In [8]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=40.0, dual=False, penalty="l2", random_state=42)

### Create the PMML

In [9]:
pipeline = PMMLPipeline([("scale",scaler),("classifier",clf)])

In [10]:
pipeline.fit(train,train_labels);

c:\users\user\desktop\test-jupyter\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
c:\users\user\desktop\test-jupyter\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
sklearn2pmml(pipeline, "predict_SPIONs.pmml", with_repr = True);